In [11]:
import pandas as pd
import zipfile
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [12]:
def parse_top_level_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    return root

top_level_xml_path = './avocado-1.0.2/data/collection.xml'
top_level_root = parse_top_level_xml(top_level_xml_path)

custodians = []
for include in top_level_root.findall('.//{http://www.w3.org/2001/XInclude}include'):
    custodian_file = include.get('href')
    custodians.append(custodian_file)

# Example output
for custodian_file in custodians:
    print(f"Custodian File: {custodian_file}")

Custodian File: custodians/001.xml
Custodian File: custodians/002.xml
Custodian File: custodians/003.xml
Custodian File: custodians/004.xml
Custodian File: custodians/005.xml
Custodian File: custodians/006.xml
Custodian File: custodians/007.xml
Custodian File: custodians/008.xml
Custodian File: custodians/009.xml
Custodian File: custodians/010.xml
Custodian File: custodians/011.xml
Custodian File: custodians/012.xml
Custodian File: custodians/013.xml
Custodian File: custodians/014.xml
Custodian File: custodians/015.xml
Custodian File: custodians/016.xml
Custodian File: custodians/017.xml
Custodian File: custodians/018.xml
Custodian File: custodians/019.xml
Custodian File: custodians/020.xml
Custodian File: custodians/021.xml
Custodian File: custodians/022.xml
Custodian File: custodians/023.xml
Custodian File: custodians/024.xml
Custodian File: custodians/025.xml
Custodian File: custodians/026.xml
Custodian File: custodians/027.xml
Custodian File: custodians/028.xml
Custodian File: cust

In [28]:
import os
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

def parse_custodian_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        return root
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except ET.ParseError as e:
        print(f"Error parsing file {file_path}: {e}")
        return None

def extract_metadata(custodian_root):
    emails = []
    if custodian_root is not None:
        items = custodian_root.findall('.//item[@type="email"]')
        for item in items:
            email_data = {field.get('name'): field.text.strip() if field.text else '' for field in item.findall('.//metadata/field')}
            body_file_path = item.find('.//file').get('path', '') if item.find('.//file') is not None else ''
            email_data['body_file_path'] = body_file_path
            emails.append(email_data)
    return emails

def read_body_file_from_zip(zip_file_path, body_file_path):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            normalized_path = body_file_path.lstrip('text/')
            if normalized_path in zip_ref.namelist():
                with zip_ref.open(normalized_path) as body_file:
                    return body_file.read().decode('utf-8')
            else:
                print(f"File {normalized_path} not found in zip {zip_file_path}")
                return ''
    except FileNotFoundError:
        print(f"Zip file not found: {zip_file_path}")
        return ''
    except Exception as e:
        print(f"Error reading {body_file_path} from {zip_file_path}: {e}")
        return ''

# Base path where XML and zip files are located
base_path = 'avocado-1.0.2/data/'
custodian_folder = 'custodians/'
text_folder = 'text/'

# List all custodian XML files
custodian_path = os.path.join(base_path, custodian_folder)
text_path = os.path.join(base_path, text_folder)
xml_files = [f for f in os.listdir(custodian_path) if f.endswith('.xml')]

# CSV file to write the emails incrementally
csv_file_path = 'emails_new.csv'

# Function to write email data to CSV
def write_emails_to_csv(emails, file_path, mode='a', header=False):
    df = pd.DataFrame(emails)
    df.to_csv(file_path, mode=mode, header=header, index=False)

first_write = True

# Process each XML file with tqdm progress bar
for xml_file in tqdm(xml_files, desc="Processing Custodian XML Files"):
    tqdm.write(f"Currently processing: {xml_file}")
    xml_path = os.path.join(custodian_path, xml_file)
    
    # Parse XML and extract metadata
    custodian_root = parse_custodian_xml(xml_path)
    if custodian_root is None:
        continue
    emails = extract_metadata(custodian_root)
    
    # Determine corresponding zip file
    zip_file_name = xml_file.replace('.xml', '.zip')
    zip_file_path = os.path.join(text_path, zip_file_name)
    
    if os.path.exists(zip_file_path):
        # Add body content to email data with tqdm progress bar
        for email in tqdm(emails, desc=f"Processing Emails in {xml_file}", leave=False):
            body_file_path = email.get('body_file_path')
            if body_file_path:
                email['body_content'] = read_body_file_from_zip(zip_file_path, body_file_path)
            else:
                email['body_content'] = ''
        
        # Write emails to CSV
        write_emails_to_csv(emails, csv_file_path, mode='a', header=first_write)
        if first_write:
            first_write = False
    else:
        print(f"Zip file not found for XML: {zip_file_path}")


Processing Custodian XML Files:   0%|          | 0/279 [00:00<?, ?it/s]

Currently processing: 162.xml


Processing Custodian XML Files:   0%|          | 1/279 [01:34<7:18:28, 94.63s/it]

Currently processing: 176.xml


Processing Custodian XML Files:   0%|          | 1/279 [01:34<7:18:28, 94.63s/it]

...

Processing Custodian XML Files:  99%|█████████▉| 277/279 [17:18:21<01:22, 41.32s/it]

Currently processing: 153.xml


Processing Custodian XML Files: 100%|█████████▉| 278/279 [17:18:21<00:29, 29.88s/it]

Currently processing: 147.xml


Processing Custodian XML Files: 100%|██████████| 279/279 [17:19:31<00:00, 223.55s/it]
